In [227]:
# relevant imports
import numpy as np
from scipy import io
import matplotlib.pyplot as plt
from matplotlib import rcParams
import rsatoolbox
import rsatoolbox.data as rsd # abbreviation to deal with dataset
import rsatoolbox.rdm as rsr
import os
import seaborn as sns
import sklearn as sk
import math
import pandas as pd
import pickle
import copy
import nibabel as nib
import re

In [228]:
# change the working directory to be the timecourse data
#os.chdir('/mnt/Data1/linjdata1/vswmda/scan_data/rsa/full_GLM_mgs_0.05_50/')
os.chdir('/gpfs/scratch/linjjiang/scan_data/rsa/full_GLM_mgs_topN/')
#os.chdir('/gpfs/scratch/linjjiang/scan_data/rsa/full_GLM_atlas_roi/')

In [229]:
subjects = ['f09','f10','f11','f12','f15','f16','f17','f18','f19']
epochs = ['delay','response','stimulus']
exp1_subjects = ['f09','f10','f11','f12','f15','f16']
exp2_subjects = ['f17','f18','f19']

In [230]:
# get ROI name
# # if full_GLM_atlas_roi
# order = ['area4-ju50',        
#           'v1-wang25','v2-wang25',
#           'ips0-wang15','ips1-wang15','ips2-wang15','ips3-wang15',
#           'ips4-wang15','ips5-wang15','spl1-wang15','ips-wang15',
#          'fef-wang25','spcs-md','ipcs-md','pmfg-md','amfg-md','ifg-md'
#           ] 
#          #'area8-hcp','area9-hcp','area9|46-hcp','area44|45|47l-hcp','fef-hcp',


# # for atlas_roi:
# order_full = ['area4-ju50',        
#           'v1-wang25','v2-wang25',
#           'ips0-wang15','ips1-wang15','ips2-wang15','ips3-wang15',
#           'ips4-wang15','ips5-wang15','spl1-wang15','ips-wang15',
#          'fef-wang25','spcs-md','ipcs-md','pmfg-md','amfg-md','ifg-md',
#               'area8-hcp','area9-hcp','area9|46-hcp','area44|45|47l-hcp','fef-hcp'
#           ] 
#          #'area8-hcp','area9-hcp','area9|46-hcp','area44|45|47l-hcp','fef-hcp',
    
    
# if full_GLM_mgs_xxx
order = ['area4', 'v1', 'v2', 'ips', 'fef', 'sfg', 'mfg', 'ifg',
         'ips0', 'ips1', 'ips2', 'ips3', 'ips4', 'ips5', 'spl1']
# for mgs roi:
order_full = order

In [231]:
# change the following as needed
roi_labels = {'area4': 'M1', 'v1': 'V1', 'v2': 'V2', 
              
              'ips0': 'IPS0', 'ips1': 'IPS1', 'ips2': 'IPS2', 
              'ips3': 'IPS3', 'ips4': 'IPS4', 'ips5': 'IPS5', 'spl1': 'SPL1',
              
          'ips': 'IPS', 'fef': 'FEF', 'sfg': 'SFG', 
          'mfg': 'MFG', 'ifg': 'IFG'}

# roi_labels = {'area4-ju50': 'M1-Julich', 'v1-wang25': 'V1-Wang25', 'v2-wang25': 'V2-Wang25', 
              
#               'ips0-wang15': 'IPS0-Wang15', 'ips1-wang15': 'IPS1-Wang15', 'ips2-wang15': 'IPS2-Wang15', 
#               'ips3-wang15': 'IPS3-Wang15', 'ips4-wang15': 'IPS4-Wang15', 'ips5-wang15': 'IPS5-Wang15', 
#               'spl1-wang15': 'SPL1-Wang15',
              
#           'ips-wang15': 'IPS-Wang15', 'fef-wang25': 'FEF-Wang25', 'spcs-md': 'sPCS-MD', 
#           'ipcs-md': 'iPCS-MD', 'amfg-md': 'aMFG-MD', 'pmfg-md': 'pMFG-MD','ifg-md': 'IFG-MD'}

# roi_order = list(roi_labels.values())
subj_labels = {index: subject for index, subject in enumerate(subjects, start=0)}
epoch_labels = {0: 'stimulus', 1: 'delay', 2: 'response'}

In [232]:
with open('rdm_crossnobis_split_by_sess_all.pkg','rb') as f:
    RDM = pickle.load(f)

In [233]:
dis_val_all = pd.DataFrame()
      
for rdm in RDM:
    subj = rdm.rdm_descriptors['subj']
    roi = rdm.rdm_descriptors['roi']
    session = rdm.rdm_descriptors['session']+1
    epoch = rdm.rdm_descriptors['epoch']
    conds = rdm.pattern_descriptors['conds']
    #print(subj,roi,session,epoch)


    a = rdm.get_matrices()[0]
    val = np.nanmean(a[np.where(~np.eye(a.shape[0],dtype=bool))])

#     # for L-R
#     labels = [x[-1] for x in conds]
#     rows_L = [i for i, label in enumerate(labels) if label == 'L']
#     cols_R = [i for i, label in enumerate(labels) if label == 'R']
#     val_LR = np.nanmean(a[np.ix_(rows_L, cols_R)])

#     # for W-T
#     labels = [x[-2] for x in conds]
#     rows_W = [i for i, label in enumerate(labels) if label == 'W']
#     cols_T = [i for i, label in enumerate(labels) if label == 'T']
#     val_WT = np.nanmean(a[np.ix_(rows_W, cols_T)])

#     # for ecc or CN
#     labels = [x[0] for x in conds]
#     rows_3_C = [i for i, label in enumerate(labels) if (label == '3') | (label == 'C')]
#     cols_5_N = [i for i, label in enumerate(labels) if (label == '5') | (label == 'N')]
#     val_ecc_or_CN = np.nanmean(a[np.ix_(rows_3_C, cols_5_N)])

    indices = np.array([[0, 2], [1, 3], [4, 6], [5, 7]])
    val_LR = np.nanmean([a[row, col] for row, col in indices])
    
    indices = np.array([[0, 1], [2, 3], [4, 5], [6, 7]])
    val_WT = np.nanmean([a[row, col] for row, col in indices])
    
    indices = np.array([[0, 4], [1, 5], [2, 6], [3, 7]])
    val_ecc = np.nanmean([a[row, col] for row, col in indices])
    
    indices = np.array([[0, 3], [1, 2], [4, 7], [5, 6]])
    val_LRWT = np.nanmean([a[row, col] for row, col in indices])
    
    indices = np.array([[0, 5], [1, 4], [2, 7], [3, 6]])
    val_eccWT = np.nanmean([a[row, col] for row, col in indices])
    
    indices = np.array([[0, 6], [1, 7], [2, 4], [3, 5]])
    val_eccLR = np.nanmean([a[row, col] for row, col in indices])
    
    indices = np.array([[0, 7], [1, 6], [2, 5], [3, 4]])
    val_all = np.nanmean([a[row, col] for row, col in indices]) 
    

    if subj[0] in exp1_subjects:
        df = pd.DataFrame({'rdm':[a],'data':[val],'data_LR':[val_LR],'data_WT':[val_WT],'data_ecc':[val_ecc],
                           'data_LRWT':[val_LRWT],'data_eccWT':[val_eccWT],'data_eccLR':[val_eccLR],'data_eccLRWT':[val_all],
                           'subj':subj,'roi':roi,'session':['Session '+str(session)],'epoch':[epoch]})
    else:
        df = pd.DataFrame({'rdm':[a],'data':[val],'data_LR':[val_LR],'data_WT':[val_WT],'data_CN':[val_ecc],
                            'data_LRWT':[val_LRWT],'data_CNWT':[val_eccWT],'data_CNLR':[val_eccLR],'data_CNLRWT':[val_all],
                           'subj':subj,'roi':roi,'session':['Session '+str(session)],'epoch':[epoch]})                
    dis_val_all = pd.concat([dis_val_all,df],axis=0)


In [234]:
print(dis_val_all)

                                                  rdm      data   data_LR  \
0   [[0.0, -0.7185203021843019, -0.051159177696733...  0.110024 -0.653435   
0   [[0.0, 0.515404632403981, 0.4463503047624071, ...  0.104163 -0.810780   
0   [[0.0, -1.7354505978536754, 1.0910847536610655... -0.359029 -1.036116   
0   [[0.0, 0.7894162058016149, 0.42766543268280177...  0.652943  0.272647   
0   [[0.0, 0.33821724657395613, 0.3018660694923435...  0.610283  0.215844   
..                                                ...       ...       ...   
0   [[0.0, 0.13163712188936721, -0.140315546387606...  0.103853  0.566828   
0   [[0.0, 0.6202412702474316, -0.4446611510966108...  0.291066 -0.002447   
0   [[0.0, 0.7585882834200403, 0.24350805510250426...  0.328333  0.931709   
0   [[0.0, -0.4471013980631687, 0.7090406147347981...  0.528133  0.390604   
0   [[0.0, 0.08690515480698868, -0.644264617480520... -0.039736  0.261492   

     data_WT  data_ecc  data_LRWT  data_eccWT  data_eccLR  data_eccLRWT sub

In [235]:
dis_val_all['roi'] = dis_val_all['roi'].map(roi_labels)
dis_val_all.index = range(len(dis_val_all))

In [236]:
from scipy.stats import ttest_1samp
from statsmodels.stats.multitest import multipletests
import warnings

def ttest_crossnobis(df,colname):
    warnings.filterwarnings("ignore")
    # Assuming your dataframe is named dis_val_all

    # Define the df, epochs, and ROIs you want to analyze
    sessions = df['session'].unique()
    epochs = ['stimulus','delay'] #dis_val_all['epoch'].unique()
    rois = df['roi'].unique()

    results = []

    # Loop through each combination of session, epoch, and ROI
    for session in sessions:
        for epoch in epochs:
            for roi in rois:
                # Filter the dataframe
                filtered_df = df[(df['session'] == session) &
                                          (df['epoch'] == epoch) &
                                          (df['roi'] == roi)]

                # Extract the 'data' values
                data_values = filtered_df[colname].dropna()

                if len(data_values) > 0:
                    # Perform a one-sample t-test
                    t_stat, p_value = ttest_1samp(data_values, 0, alternative='greater')
                    dof = len(data_values)-1
                    
                    # Store the result
                    results.append({'session': session, 'epoch': epoch, 'roi': roi, 'mean': data_values.mean(), 
                                    't_stat': t_stat, 'p_value': p_value, 'dof': dof})

    # Convert results to a dataframe
    results_df = pd.DataFrame(results)
    
#     # Correct for multiple comparisons
#     p_values = results_df['p_value'].values.flatten()
#     corrected_p_values = multipletests(p_values, method='fdr_bh')[1]
#     results_df['p_value'] = corrected_p_values.reshape(-1, 1)
    
    # Filter the results to only include rows with p_value less than 0.05
    significant_results_df = results_df[results_df['p_value'] < 0.05]

    # Define custom order for epochs
    epoch_order = ['stimulus', 'delay']

    # Convert 'epoch' column to a categorical type with the custom order
    significant_results_df['epoch'] = pd.Categorical(significant_results_df['epoch'], categories=epoch_order, ordered=True)

    # Sort by the custom epoch order
    significant_results_df = significant_results_df.sort_values(by=['session', 'epoch'])

    # Print the significant results
    return results_df,significant_results_df


In [237]:
from scipy.stats import ttest_rel
import pandas as pd
from statsmodels.stats.multitest import multipletests

def ttest_paired_sessions(dis_val_all, colname, session1, session2):
    warnings.filterwarnings("ignore")
    # Define the epochs and ROIs you want to analyze
    epochs = ['stimulus','delay'] #dis_val_all['epoch'].unique()
    rois = dis_val_all['roi'].unique()

    results = []

    # Loop through each combination of epoch and ROI
    for epoch in epochs:
        for roi in rois:
            # Filter the dataframe for each session
            df1 = dis_val_all[(dis_val_all['session'] == session1) &
                                        (dis_val_all['epoch'] == epoch) &
                                        (dis_val_all['roi'] == roi)]#[colname].dropna()

            df2 = dis_val_all[(dis_val_all['session'] == session2) &
                                        (dis_val_all['epoch'] == epoch) &
                                        (dis_val_all['roi'] == roi)]#[colname].dropna()

            
            # Identify common subject IDs
            common_subject_ids = set(df1['subj']).intersection(set(df2['subj']))

            # Filter dataframes to include only common subject IDs
            df1_filtered = df1[df1['subj'].isin(common_subject_ids)]
            df2_filtered = df2[df2['subj'].isin(common_subject_ids)]

            # Merge the filtered dataframes on 'subject_id'
            merged_df = pd.merge(df1_filtered, df2_filtered, on='subj', suffixes=('_df1', '_df2'))

            # Perform the paired t-test
            t_stat, p_value = ttest_rel(merged_df[colname+'_df1'].dropna(), merged_df[colname+'_df2'].dropna())
            dof = len(merged_df[colname+'_df1'])-1
            
#             # Ensure that data lengths match for paired test
#             min_length = min(len(data_session1), len(data_session2))
#             if min_length > 0:
#                 # Perform a paired t-test
#                 t_stat, p_value, df, cint = ttest_rel(data_session1[:min_length], data_session2[:min_length])

            # Store the result
            results.append({'epoch': epoch, 'roi': roi, 'model': colname,
                            'session1-mean': merged_df[colname+'_df1'].mean(skipna=True),
                            'session2-mean': merged_df[colname+'_df2'].mean(skipna=True),
                             'Mean difference': merged_df[colname+'_df1'].mean(skipna=True)-merged_df[colname+'_df2'].mean(skipna=True),
                            't_stat': t_stat, 'p_value': p_value,
                            'dof': dof})

    # Convert results to a dataframe
    results_df = pd.DataFrame(results)

#     # Correct for multiple comparisons
#     p_values = results_df['p_value'].values.flatten()
#     corrected_p_values = multipletests(p_values, method='fdr_bh')[1]
#     results_df['p_value'] = corrected_p_values.reshape(-1, 1)
    
    # Filter the results to only include rows with p_value less than 0.05
    significant_results_df = results_df[results_df['p_value'] < 0.05]

    # Define custom order for epochs
    epoch_order = ['stimulus', 'delay']#, 'response']

    # Convert 'epoch' column to a categorical type with the custom order
    significant_results_df['epoch'] = pd.Categorical(significant_results_df['epoch'], categories=epoch_order, ordered=True)

    # Sort by the custom epoch order
    significant_results_df = significant_results_df.sort_values(by='epoch')

    # Print the significant results
    return results_df,significant_results_df


In [238]:
import warnings
def multi_corr(result_df):
    warnings.filterwarnings('ignore')
    # Correct for multiple comparisons
    p_values = result_df['p_value'].values.flatten()
    corrected_p_values = multipletests(p_values, method='fdr_bh')[1]
    result_df['p_value_corr'] = corrected_p_values.reshape(-1, 1)
    
    return result_df

In [239]:
df_temp = pd.DataFrame()
df_temp_sess = pd.DataFrame()

In [240]:
selected_rois = ['FEF','SFG','MFG','IFG']
selected_rois_key = ['fef','sfg','mfg','ifg']
selected_epochs = ['stimulus','delay']
selected_epochs_key = [0,1]
df = dis_val_all[(dis_val_all['roi'].isin(selected_rois)) 
                  & (dis_val_all['epoch'].isin(selected_epochs))]

selected_roi_labels = {key: roi_labels[key] for key in selected_rois_key if key in roi_labels}
selected_epoch_labels = {key: epoch_labels[key] for key in selected_epochs_key if key in epoch_labels}

In [241]:
print(df)

                                                   rdm      data   data_LR  \
4    [[0.0, 0.33821724657395613, 0.3018660694923435...  0.610283  0.215844   
5    [[0.0, 1.2308863240202832, 0.47414633178543286...  0.235184 -0.509989   
6    [[0.0, 0.7672158532558693, 0.16881289559011936...  0.517063 -0.028731   
7    [[0.0, 0.007909335520818572, 0.412265847597252...  0.206964 -0.347950   
23   [[0.0, -0.14623296794335092, -0.46155595667054...  0.032027 -0.055442   
..                                                 ...       ...       ...   
746  [[0.0, 0.3435374562788888, 0.04297877721743861...  0.184197  0.314613   
747  [[0.0, 0.28095945313034076, -0.122688203558198...  0.159931 -0.032291   
748  [[0.0, 0.3478346273050898, -0.7453488073505579...  0.170336  0.196071   
749  [[0.0, 0.3894990386427912, 0.01440588431735271...  0.195534  0.173342   
750  [[0.0, 0.07144095221787151, -0.438381060404818...  0.100733  0.232983   

      data_WT  data_ecc  data_LRWT  data_eccWT  data_eccLR  dat

In [242]:
result_df, sig_df = ttest_crossnobis(df,'data_LR')
print(sig_df)

result_df = multi_corr(result_df)
sig_df = result_df[result_df['p_value'] < 0.05]
print(sig_df)

df_temp = pd.concat((df_temp,result_df),axis=0)

Empty DataFrame
Columns: [session, epoch, roi, mean, t_stat, p_value, dof]
Index: []
Empty DataFrame
Columns: [session, epoch, roi, mean, t_stat, p_value, dof, p_value_corr]
Index: []


In [243]:
result_df1,sig_df1 = ttest_paired_sessions(df, 'data_LR', 'Session 1', 'Session 2')
print(sig_df1)

result_df1 = multi_corr(result_df1)
df_temp_sess = pd.concat((df_temp_sess,result_df1),axis=0)
#print(result_df1)

Empty DataFrame
Columns: [epoch, roi, model, session1-mean, session2-mean, Mean difference, t_stat, p_value, dof]
Index: []


In [244]:
result_df, sig_df = ttest_crossnobis(df,'data_WT')
print(sig_df)

result_df = multi_corr(result_df)
sig_df = result_df[result_df['p_value'] < 0.05]
print(sig_df)
df_temp = pd.concat((df_temp,result_df),axis=0)

     session     epoch  roi      mean    t_stat   p_value  dof
3  Session 1  stimulus  IFG  0.215704  1.871583  0.049084    8
     session     epoch  roi      mean    t_stat   p_value  dof  p_value_corr
3  Session 1  stimulus  IFG  0.215704  1.871583  0.049084    8      0.441625


In [245]:
result_df1,sig_df1 = ttest_paired_sessions(df, 'data_WT', 'Session 1', 'Session 2')
print(sig_df1)

result_df1 = multi_corr(result_df1)
df_temp_sess = pd.concat((df_temp_sess,result_df1),axis=0)

      epoch  roi    model  session1-mean  session2-mean  Mean difference  \
3  stimulus  IFG  data_WT        0.28137      -0.202803         0.484173   

     t_stat   p_value  dof  
3  2.682135  0.031442    7  


In [246]:
result_df, sig_df = ttest_crossnobis(df,'data_ecc')
print(sig_df)

result_df = multi_corr(result_df)
sig_df = result_df[result_df['p_value'] < 0.05]
print(sig_df)
df_temp = pd.concat((df_temp,result_df),axis=0)

      session  epoch  roi      mean    t_stat   p_value  dof
15  Session 2  delay  IFG  0.437243  2.702899  0.026966    4
      session  epoch  roi      mean    t_stat   p_value  dof  p_value_corr
15  Session 2  delay  IFG  0.437243  2.702899  0.026966    4      0.431457


In [247]:
result_df1,sig_df1 = ttest_paired_sessions(df, 'data_ecc', 'Session 1', 'Session 2')
print(sig_df1)

result_df1 = multi_corr(result_df1)
df_temp_sess = pd.concat((df_temp_sess,result_df1),axis=0)

   epoch  roi     model  session1-mean  session2-mean  Mean difference  \
7  delay  IFG  data_ecc       0.015007       0.437243        -0.422236   

     t_stat   p_value  dof  
7 -3.545318  0.023899    7  


In [248]:
result_df, sig_df = ttest_crossnobis(df,'data_CN')
print(sig_df)

result_df = multi_corr(result_df)
sig_df = result_df[result_df['p_value'] < 0.05]
print(sig_df)
df_temp = pd.concat((df_temp,result_df),axis=0)

     session     epoch  roi      mean     t_stat   p_value  dof
2  Session 1  stimulus  MFG  0.284410  14.311811  0.002423    2
8  Session 2  stimulus  FEF  0.107569   3.986022  0.028779    2
9  Session 2  stimulus  SFG  0.204812   3.736029  0.032381    2
     session     epoch  roi      mean     t_stat   p_value  dof  p_value_corr
2  Session 1  stimulus  MFG  0.284410  14.311811  0.002423    2      0.038773
8  Session 2  stimulus  FEF  0.107569   3.986022  0.028779    2      0.172698
9  Session 2  stimulus  SFG  0.204812   3.736029  0.032381    2      0.172698


In [249]:
result_df1,sig_df1 = ttest_paired_sessions(df, 'data_CN', 'Session 1', 'Session 2')
print(sig_df1)

result_df1 = multi_corr(result_df1)
df_temp_sess = pd.concat((df_temp_sess,result_df1),axis=0)

      epoch  roi    model  session1-mean  session2-mean  Mean difference  \
0  stimulus  FEF  data_CN      -0.065068       0.107569        -0.172637   
2  stimulus  MFG  data_CN       0.284410      -0.039902         0.324312   
7     delay  IFG  data_CN       0.339114      -0.012521         0.351635   

      t_stat   p_value  dof  
0 -32.310156  0.000957    7  
2   6.031203  0.026407    7  
7   4.472212  0.046536    7  


In [250]:
selected_rois = ['IPS0','IPS1','IPS2','SPL1'] #'IPS3','IPS4','IPS5',
selected_rois_key = ['ips0','ips1','ips2','spl1'] #'ips3','ips4','ips5',dpi=1200
selected_epochs = ['stimulus','delay']
selected_epochs_key = [0,1]
df = dis_val_all[(dis_val_all['roi'].isin(selected_rois)) 
                  & (dis_val_all['epoch'].isin(selected_epochs))]

selected_roi_labels = {key: roi_labels[key] for key in selected_rois_key if key in roi_labels}
selected_epoch_labels = {key: epoch_labels[key] for key in selected_epochs_key if key in epoch_labels}

In [251]:
print(df)

                                                   rdm      data   data_LR  \
8    [[0.0, 1.933391942440123, 0.7752352199817576, ...  0.592165  0.149298   
9    [[0.0, 1.2393029025305085, -0.5708026373825937...  0.196745 -0.137203   
10   [[0.0, 0.8422746085769184, 0.6507649215205681,...  1.151673  0.769467   
14   [[0.0, -0.531496744710929, -0.4205932490351345...  0.109421 -0.143819   
31   [[0.0, 0.44873949681896014, -0.647079307232857...  0.594924  0.115622   
..                                                 ...       ...       ...   
754  [[0.0, -0.20136493939048408, -0.67225812789134...  0.273156  0.682958   
755  [[0.0, 0.20303410279739734, -0.635741411255692...  0.523121  0.575477   
756  [[0.0, -0.1953592955010104, -0.797026035922329... -0.139339  0.203384   
763  [[0.0, -0.4471013980631687, 0.7090406147347981...  0.528133  0.390604   
764  [[0.0, 0.08690515480698868, -0.644264617480520... -0.039736  0.261492   

      data_WT  data_ecc  data_LRWT  data_eccWT  data_eccLR  dat

In [252]:
result_df, sig_df = ttest_crossnobis(df,'data_LR')
print(sig_df)

result_df = multi_corr(result_df)
sig_df = result_df[result_df['p_value'] < 0.05]
print(sig_df)

df_temp = pd.concat((df_temp,result_df),axis=0)

     session     epoch   roi      mean    t_stat   p_value  dof
8  Session 2  stimulus  IPS0  0.233931  2.269108  0.028777    7
     session     epoch   roi      mean    t_stat   p_value  dof  p_value_corr
8  Session 2  stimulus  IPS0  0.233931  2.269108  0.028777    7      0.304604


In [253]:
result_df1,sig_df1 = ttest_paired_sessions(df, 'data_LR', 'Session 1', 'Session 2')
print(sig_df1)

result_df1 = multi_corr(result_df1)
df_temp_sess = pd.concat((df_temp_sess,result_df1),axis=0)

Empty DataFrame
Columns: [epoch, roi, model, session1-mean, session2-mean, Mean difference, t_stat, p_value, dof]
Index: []


In [254]:
result_df, sig_df = ttest_crossnobis(df,'data_WT')
print(sig_df)

result_df = multi_corr(result_df)
sig_df = result_df[result_df['p_value'] < 0.05]
print(sig_df)

df_temp = pd.concat((df_temp,result_df),axis=0)

      session     epoch   roi      mean    t_stat   p_value  dof
2   Session 1  stimulus  IPS2  0.345728  2.265131  0.026647    8
4   Session 1     delay  IPS0  0.485110  3.427808  0.004492    8
7   Session 1     delay  SPL1  0.241622  1.865223  0.049566    8
15  Session 2     delay  SPL1  0.405831  2.194512  0.032125    7
      session     epoch   roi      mean    t_stat   p_value  dof  p_value_corr
2   Session 1  stimulus  IPS2  0.345728  2.265131  0.026647    8      0.171332
4   Session 1     delay  IPS0  0.485110  3.427808  0.004492    8      0.071871
7   Session 1     delay  SPL1  0.241622  1.865223  0.049566    8      0.198264
15  Session 2     delay  SPL1  0.405831  2.194512  0.032125    7      0.171332


In [255]:
result_df1,sig_df1 = ttest_paired_sessions(df, 'data_WT', 'Session 1', 'Session 2')
print(sig_df1)

result_df1 = multi_corr(result_df1)
df_temp_sess = pd.concat((df_temp_sess,result_df1),axis=0)

Empty DataFrame
Columns: [epoch, roi, model, session1-mean, session2-mean, Mean difference, t_stat, p_value, dof]
Index: []


In [256]:
result_df, sig_df = ttest_crossnobis(df,'data_ecc')
print(sig_df)

result_df = multi_corr(result_df)
sig_df = result_df[result_df['p_value'] < 0.05]
print(sig_df)

df_temp = pd.concat((df_temp,result_df),axis=0)

      session     epoch   roi      mean    t_stat   p_value  dof
0   Session 1  stimulus  IPS0  0.496028  3.947289  0.005440    5
12  Session 2     delay  IPS0  0.431127  4.528119  0.005296    4
13  Session 2     delay  IPS1  0.363780  2.547063  0.031751    4
15  Session 2     delay  SPL1  0.530416  3.072228  0.018606    4
      session     epoch   roi      mean    t_stat   p_value  dof  p_value_corr
0   Session 1  stimulus  IPS0  0.496028  3.947289  0.005440    5      0.043521
12  Session 2     delay  IPS0  0.431127  4.528119  0.005296    4      0.043521
13  Session 2     delay  IPS1  0.363780  2.547063  0.031751    4      0.127006
15  Session 2     delay  SPL1  0.530416  3.072228  0.018606    4      0.099232


In [257]:
result_df1,sig_df1 = ttest_paired_sessions(df, 'data_ecc', 'Session 1', 'Session 2')
print(sig_df1)

result_df1 = multi_corr(result_df1)
df_temp_sess = pd.concat((df_temp_sess,result_df1),axis=0)

Empty DataFrame
Columns: [epoch, roi, model, session1-mean, session2-mean, Mean difference, t_stat, p_value, dof]
Index: []


In [258]:
result_df, sig_df = ttest_crossnobis(df,'data_CN')
print(sig_df)

result_df = multi_corr(result_df)
sig_df = result_df[result_df['p_value'] < 0.05]
print(sig_df)

df_temp = pd.concat((df_temp,result_df),axis=0)

Empty DataFrame
Columns: [session, epoch, roi, mean, t_stat, p_value, dof]
Index: []
Empty DataFrame
Columns: [session, epoch, roi, mean, t_stat, p_value, dof, p_value_corr]
Index: []


In [259]:
result_df1,sig_df1 = ttest_paired_sessions(df, 'data_CN', 'Session 1', 'Session 2')
print(sig_df1)

result_df1 = multi_corr(result_df1)
df_temp_sess = pd.concat((df_temp_sess,result_df1),axis=0)

Empty DataFrame
Columns: [epoch, roi, model, session1-mean, session2-mean, Mean difference, t_stat, p_value, dof]
Index: []


In [260]:
selected_rois = ['V1','V2']
selected_rois_key = ['v1','v2']
selected_epochs = ['stimulus','delay']
selected_epochs_key = [0,1]
df = dis_val_all[(dis_val_all['roi'].isin(selected_rois)) 
                  & (dis_val_all['epoch'].isin(selected_epochs))]

selected_roi_labels = {key: roi_labels[key] for key in selected_rois_key if key in roi_labels}
selected_epoch_labels = {key: epoch_labels[key] for key in selected_epochs_key if key in epoch_labels}

In [261]:
print(df)

                                                   rdm      data   data_LR  \
1    [[0.0, 0.515404632403981, 0.4463503047624071, ...  0.104163 -0.810780   
2    [[0.0, -1.7354505978536754, 1.0910847536610655... -0.359029 -1.036116   
17   [[0.0, 0.08040501907406696, -0.736327484314584...  0.573809 -0.168935   
18   [[0.0, -1.126141106317723, -2.490493852902801,...  0.610885 -0.009385   
19   [[0.0, 1.056061172409632, -0.8275477466960791,...  0.436814 -0.519577   
..                                                 ...       ...       ...   
710  [[0.0, 0.2123288287426872, -0.6419979192252229...  0.297121 -0.487780   
737  [[0.0, 0.4290929092280638, 0.23815449794154428...  0.264372  0.099424   
738  [[0.0, 0.5738339770747108, -0.5562865821345161...  0.480638  0.398122   
739  [[0.0, 0.6377790916932221, 0.1301041536118579,...  0.366800  0.158779   
740  [[0.0, 0.21682605898915155, -0.096091504020221...  0.411087  0.365988   

      data_WT  data_ecc  data_LRWT  data_eccWT  data_eccLR  dat

In [262]:
result_df, sig_df = ttest_crossnobis(df,'data_LR')
print(sig_df)

result_df = multi_corr(result_df)
sig_df = result_df[result_df['p_value'] < 0.05]
print(sig_df)

df_temp = pd.concat((df_temp,result_df),axis=0)

     session     epoch roi      mean    t_stat   p_value  dof
1  Session 1  stimulus  V2  0.221704  3.008561  0.008425    8
4  Session 2  stimulus  V1  0.427819  2.284693  0.028123    7
5  Session 2  stimulus  V2  0.480862  2.200842  0.031826    7
     session     epoch roi      mean    t_stat   p_value  dof  p_value_corr
1  Session 1  stimulus  V2  0.221704  3.008561  0.008425    8      0.067402
4  Session 2  stimulus  V1  0.427819  2.284693  0.028123    7      0.084869
5  Session 2  stimulus  V2  0.480862  2.200842  0.031826    7      0.084869


In [263]:
result_df1,sig_df1 = ttest_paired_sessions(df, 'data_LR', 'Session 1', 'Session 2')
print(sig_df1)

result_df1 = multi_corr(result_df1)
df_temp_sess = pd.concat((df_temp_sess,result_df1),axis=0)

Empty DataFrame
Columns: [epoch, roi, model, session1-mean, session2-mean, Mean difference, t_stat, p_value, dof]
Index: []


In [264]:
result_df, sig_df = ttest_crossnobis(df,'data_WT')
print(sig_df)

result_df = multi_corr(result_df)
sig_df = result_df[result_df['p_value'] < 0.05]
print(sig_df)

df_temp = pd.concat((df_temp,result_df),axis=0)

     session     epoch roi      mean    t_stat   p_value  dof
0  Session 1  stimulus  V1  0.882789  3.616216  0.003410    8
1  Session 1  stimulus  V2  1.094411  3.910349  0.002239    8
4  Session 2  stimulus  V1  1.015191  3.630230  0.004198    7
5  Session 2  stimulus  V2  1.539330  4.188193  0.002048    7
     session     epoch roi      mean    t_stat   p_value  dof  p_value_corr
0  Session 1  stimulus  V1  0.882789  3.616216  0.003410    8      0.008395
1  Session 1  stimulus  V2  1.094411  3.910349  0.002239    8      0.008395
4  Session 2  stimulus  V1  1.015191  3.630230  0.004198    7      0.008395
5  Session 2  stimulus  V2  1.539330  4.188193  0.002048    7      0.008395


In [265]:
result_df1,sig_df1 = ttest_paired_sessions(df, 'data_WT', 'Session 1', 'Session 2')
print(sig_df1)

result_df1 = multi_corr(result_df1)
df_temp_sess = pd.concat((df_temp_sess,result_df1),axis=0)

Empty DataFrame
Columns: [epoch, roi, model, session1-mean, session2-mean, Mean difference, t_stat, p_value, dof]
Index: []


In [266]:
result_df, sig_df = ttest_crossnobis(df,'data_ecc')
print(sig_df)

result_df = multi_corr(result_df)
sig_df = result_df[result_df['p_value'] < 0.05]
print(sig_df)

df_temp = pd.concat((df_temp,result_df),axis=0)

     session     epoch roi      mean    t_stat   p_value  dof
0  Session 1  stimulus  V1  1.044870  4.563646  0.003018    5
1  Session 1  stimulus  V2  1.644306  8.747914  0.000162    5
4  Session 2  stimulus  V1  1.180028  6.776236  0.001238    4
5  Session 2  stimulus  V2  2.360088  5.575677  0.002536    4
     session     epoch roi      mean    t_stat   p_value  dof  p_value_corr
0  Session 1  stimulus  V1  1.044870  4.563646  0.003018    5      0.006037
1  Session 1  stimulus  V2  1.644306  8.747914  0.000162    5      0.001294
4  Session 2  stimulus  V1  1.180028  6.776236  0.001238    4      0.004951
5  Session 2  stimulus  V2  2.360088  5.575677  0.002536    4      0.006037


In [267]:
result_df1,sig_df1 = ttest_paired_sessions(df, 'data_ecc', 'Session 1', 'Session 2')
print(sig_df1)

result_df1 = multi_corr(result_df1)
df_temp_sess = pd.concat((df_temp_sess,result_df1),axis=0)

      epoch roi     model  session1-mean  session2-mean  Mean difference  \
1  stimulus  V2  data_ecc       1.537011       2.360088        -0.823077   

     t_stat   p_value  dof  
1 -3.153485  0.034399    7  


In [268]:
result_df, sig_df = ttest_crossnobis(df,'data_CN')
print(sig_df)

result_df = multi_corr(result_df)
sig_df = result_df[result_df['p_value'] < 0.05]
print(sig_df)

df_temp = pd.concat((df_temp,result_df),axis=0)

Empty DataFrame
Columns: [session, epoch, roi, mean, t_stat, p_value, dof]
Index: []
Empty DataFrame
Columns: [session, epoch, roi, mean, t_stat, p_value, dof, p_value_corr]
Index: []


In [269]:
result_df1,sig_df1 = ttest_paired_sessions(df, 'data_CN', 'Session 1', 'Session 2')
print(sig_df1)

result_df1 = multi_corr(result_df1)
df_temp_sess = pd.concat((df_temp_sess,result_df1),axis=0)

Empty DataFrame
Columns: [epoch, roi, model, session1-mean, session2-mean, Mean difference, t_stat, p_value, dof]
Index: []


In [270]:
print(df_temp)

      session     epoch  roi      mean    t_stat   p_value  dof  p_value_corr
0   Session 1  stimulus  FEF -0.045245 -0.688792  0.744784    8      0.851182
1   Session 1  stimulus  SFG  0.111293  1.372762  0.103537    8      0.497697
2   Session 1  stimulus  MFG  0.066785  0.517740  0.309324    8      0.707027
3   Session 1  stimulus  IFG  0.001948  0.019861  0.492320    8      0.851182
4   Session 1     delay  FEF  0.074341  0.943021  0.186637    8      0.497697
..        ...       ...  ...       ...       ...       ...  ...           ...
3   Session 1     delay   V2  0.218864  1.201704  0.176235    2      0.772603
4   Session 2  stimulus   V1  0.164982  1.046367  0.202606    2      0.772603
5   Session 2  stimulus   V2  0.010188  0.021391  0.492438    2      0.787901
6   Session 2     delay   V1 -0.354551 -7.947000  0.992266    2      0.992266
7   Session 2     delay   V2  0.042801  0.123193  0.456609    2      0.787901

[160 rows x 8 columns]


In [271]:
print(df_temp_sess)

       epoch  roi     model  session1-mean  session2-mean  Mean difference  \
0   stimulus  FEF   data_LR      -0.046764       0.074313        -0.121077   
1   stimulus  SFG   data_LR       0.115631      -0.039931         0.155562   
2   stimulus  MFG   data_LR       0.084006       0.052657         0.031349   
3   stimulus  IFG   data_LR       0.028835      -0.054545         0.083380   
4      delay  FEF   data_LR       0.056653      -0.204449         0.261102   
..       ...  ...       ...            ...            ...              ...   
3      delay   V2  data_ecc      -0.098356       0.103030        -0.201386   
0   stimulus   V1   data_CN      -0.212976       0.164982        -0.377958   
1   stimulus   V2   data_CN      -0.238215       0.010188        -0.248403   
2      delay   V1   data_CN       0.117994      -0.354551         0.472544   
3      delay   V2   data_CN       0.218864       0.042801         0.176063   

      t_stat   p_value  dof  p_value_corr  
0  -1.558648  0.163

In [272]:
df_temp.to_csv('tbl_crossnobis_against_zero.csv')
df_temp_sess.to_csv('tbl_crossnobis_btw_sess.csv')